## extract specific classes with images

In [11]:
import numpy as np
import cv2
import os
import glob

folder = r"D:/videos/bear/data/bear_images/gray"
# folders = os.listdir(folder_path)

count = 0
target_class = str('-')

## if you want replace class number
set_class = 2
error = 0
# for folder in folders:
# path = os.path.join(folder_path, folder)
image_files = glob.glob(f'{folder}/*.jpg') + glob.glob(f'{folder}/*.png')

txt_list = []
img_list = []
for filename in image_files:
    label = filename[:filename.rfind('.')] + '.txt'
    # try:
    f = open(label, 'r')
    line = f.readlines()
    last_count = count
    if(len(line) > 0):
        for i in line:
            if(i[0] != target_class):
                #break ##?
                continue ##?
        img_path = filename
        txt_path = img_path[:img_path.find('.')] + '.txt'
        # print(filename)
        
        img = cv2.imread(filename)
        cv2.imwrite(img_path, img)
        
        for i in line:
            if(i[0] == target_class):
                count += 1
                contents = str(set_class) + i[i.find(' '):]    
                txt = open(txt_path, "w")
                txt.write(contents) 
                txt.close()

        nofound = 1
        if(last_count==count):
            nofound = 1
        else:
            nofound = 0
        
        if(nofound):
            txt_list.append(txt_path)
            img_list.append(img_path)
    f.close()
    # except:
    #     error += 1

# for i in range(len(txt_list)):
#     os.remove(txt_list[i])
#     try:
#         os.remove(img_list[i])
#     except:
#         print(img_list[i])

print("count: %d\n",count)
print("error: %d\n",error)

count: %d
 2889
error: %d
 0


# 빈 txt 파일 생성

In [7]:
import os

def create_missing_txt_files(src_folder):
    # 폴더 내 모든 파일 목록을 가져옵니다.
    files = os.listdir(src_folder)
    
    # .jpg 파일을 순회하면서 동일한 이름의 .txt 파일이 있는지 확인합니다.
    for file in files:
        if file.endswith('.jpg'):
            base_name = file[:-4]  # .jpg 확장자를 제외한 파일 이름
            txt_file = base_name + '.txt'
            txt_file_path = os.path.join(src_folder, txt_file)
            if txt_file not in files:
                try:
                    # 동일한 이름의 빈 .txt 파일을 생성합니다.
                    with open(txt_file_path, 'w') as f:
                        pass
                    # print(f"{txt_file_path} 파일이 생성되었습니다.")
                except Exception as e:
                    print(f"{txt_file_path} 파일을 생성하는 중 오류가 발생했습니다: {e}")

# 사용 예시
src_folder = "D:/videos/bear/data/new/79/black"
create_missing_txt_files(src_folder)


# 타겟 클래스 아이디 변경

In [6]:
import os
import glob
from tqdm import tqdm

folder_path = "D:/videos/bear/test/test_1"
target_class = '-'
set_class = '2'
count = 0
error = 0

# Get the list of image files in the folder
image_files = glob.glob(os.path.join(folder_path, '*.jpg'))

# Process files with a progress bar
with tqdm(total=len(image_files), desc="Processing files") as pbar:
    for filename in image_files:
        label = filename[:filename.rfind('.')] + '.txt'
        try:
            with open(label, 'r') as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = parts[0]
                    if class_id == target_class:
                        parts[0] = set_class
                        count += 1
                    new_lines.append(' '.join(parts) + '\n')
                else:
                    print(f"Invalid line found and removed in {label}: {line.strip()}")

            with open(label, "w") as txt:
                txt.writelines(new_lines)

        except Exception as e:
            error += 1
            print(f"Error processing file {filename}: {e}")

        pbar.update(1)

print(f"count: {count}")
print(f"error: {error}")


Processing files: 100%|██████████| 14935/14935 [00:08<00:00, 1785.72it/s]

count: 0
error: 0


# 라벨링 된 것들만 따로 이동 복사

In [9]:
import os
import shutil
from tqdm import tqdm

# 원본 경로와 목적지 경로 설정
source_path = 'D:/videos/bear/data/new/79'
destination_path = 'D:/videos/bear/data/new/gathered'

# 원본 경로 내의 폴더 목록 가져오기
folders = [os.path.join(source_path, folder) for folder in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, folder))]

# 모든 폴더 내의 파일 수 계산
total_files = sum([len(files) for r, d, files in os.walk(source_path) if files])

# 프로그래스 바 설정
with tqdm(total=total_files, desc="Processing files") as pbar:
    # 각 폴더를 순회하며 파일 복사
    for folder in folders:
        folder_name = os.path.basename(folder)
        for file in os.listdir(folder):
            if file.endswith('.txt'):
                txt_file_path = os.path.join(folder, file)
                jpg_file_path = os.path.join(folder, file.replace('.txt', '.jpg'))
                
                # txt 파일이 비어있지 않은 경우
                with open(txt_file_path, 'r') as txt_file:
                    content = txt_file.read().strip()
                    if content:
                        # 파일 이름에 폴더 이름 추가하여 복사
                        new_txt_file_name = f"{os.path.splitext(file)[0]}_{folder_name}.txt"
                        new_jpg_file_name = f"{os.path.splitext(file)[0]}_{folder_name}.jpg"
                        
                        new_txt_file_path = os.path.join(destination_path, new_txt_file_name)
                        new_jpg_file_path = os.path.join(destination_path, new_jpg_file_name)
                        
                        shutil.copy(txt_file_path, new_txt_file_path)
                        shutil.copy(jpg_file_path, new_jpg_file_path)
            # 파일 하나 처리 완료 후 프로그래스 바 업데이트
            pbar.update(1)


Processing files: 86757it [02:14, 644.05it/s]                             


# jpg가 없는 라벨링 값 제거

In [4]:
import os

def delete_orphan_txt_files(src_folder):
    # 폴더 내 모든 파일 목록을 가져옵니다.
    files = os.listdir(src_folder)
    
    # 모든 .txt 파일을 순회하면서 동일한 이름의 .jpg 파일이 있는지 확인합니다.
    for file in files:
        if file.endswith('.txt'):
            base_name = file[:-4]  # .txt 확장자를 제외한 파일 이름
            jpg_file = base_name + '.jpg'
            if jpg_file not in files:
                file_path = os.path.join(src_folder, file)
                try:
                    os.remove(file_path)
                    print(f"{file_path} 파일이 삭제되었습니다.")
                except Exception as e:
                    print(f"{file_path} 파일을 삭제하는 중 오류가 발생했습니다: {e}")

# 사용 예시
src_folder = 'D:/videos/bear/data/new/79/gray'
delete_orphan_txt_files(src_folder)

D:/videos/bear/data/new/79/gray\bear_00015 (60).txt 파일이 삭제되었습니다.
D:/videos/bear/data/new/79/gray\gettyimages-1425419282-640x640.txt 파일이 삭제되었습니다.


# 특정 클래스를 다른 폴더로 이동

In [ ]:
import os
import shutil

def clean_and_delete_files(src_folder, dest_folder, target_class):
    # Ensure destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Iterate over all files in the source folder
    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.txt'):
            txt_file_path = os.path.join(src_folder, file_name)
            jpg_file_path = os.path.splitext(txt_file_path)[0] + '.jpg'
            
            if not os.path.exists(jpg_file_path):
                continue

            # Read and clean the txt file
            with open(txt_file_path, 'r') as file:
                lines = file.readlines()

            valid_lines = []
            has_class_id_3 = False
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    if class_id == int(target_class):
                        has_class_id_3 = True
                    else:
                        valid_lines.append(line)
                else:
                    print(f"Invalid line found and removed in {file_name}: {line.strip()}")

            # Delete files if they contain class_id == target_class
            if has_class_id_3:
                os.remove(txt_file_path)
                os.remove(jpg_file_path)
                print(f"Deleted {file_name} and corresponding jpg from {src_folder}")
            else:
                # Otherwise, save the cleaned txt file
                with open(txt_file_path, 'w') as file:
                    file.writelines(valid_lines)

# Example usage
src_folder = r"D:/videos/bear/data/bear_images/black"
dest_folder = r"D:/videos/bear/data/bear_images/black/sep"

target_class = 3
clean_and_delete_files(src_folder, dest_folder, target_class)
